In [2]:
import mysql.connector
mydb = mysql.connector.connect(host="localhost",user="root",password="1234")
mycursor = mydb.cursor(buffered=True)


In [8]:
mycursor.execute("CREATE DATABASE IF NOT EXISTS phonepe")
mycursor.execute("USE phonepe")

In [12]:
import os
import pandas as pd
import json
import mysql.connector as sql

In [3]:
path="pulse/data/aggregated/transaction/country/india/state/"
Agg_state_list=os.listdir(path)
Agg_state_list

clm={'State':[], 'Year':[],'Quater':[],'Transacion_type':[], 'Transacion_count':[], 'Transacion_amount':[]}
for i in Agg_state_list:
    p_i=path+i+"/"
    Agg_yr=os.listdir(p_i)
    for j in Agg_yr:
        p_j=p_i+j+"/"
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+k
            Data=open(p_k,'r')
            D=json.load(Data)
            for z in D['data']['transactionData']:
              Name=z['name']
              count=z['paymentInstruments'][0]['count']
              amount=z['paymentInstruments'][0]['amount']
              clm['Transacion_type'].append(Name)
              clm['Transacion_count'].append(count)
              clm['Transacion_amount'].append(amount)
              clm['State'].append(i)
              clm['Year'].append(j)
              clm['Quater'].append(int(k.strip('.json')))
#Succesfully created a dataframe
Agg_Trans=pd.DataFrame(clm)

In [4]:
Agg_Trans

,State,Year,Quater,Transacion_type,Transacion_count,Transacion_amount
0,andaman-&-nicobar-islands,2018,1,Recharge & bill payments,4200,1.845307e+06
1,andaman-&-nicobar-islands,2018,1,Peer-to-peer payments,1871,1.213866e+07
2,andaman-&-nicobar-islands,2018,1,Merchant payments,298,4.525072e+05
3,andaman-&-nicobar-islands,2018,1,Financial Services,33,1.060142e+04
4,andaman-&-nicobar-islands,2018,1,Others,256,1.846899e+05
...,...,...,...,...,...,...
4129,west-bengal,2023,3,Merchant payments,296965002,2.004404e+11
4130,west-bengal,2023,3,Peer-to-peer payments,271150280,8.310501e+11
4131,west-bengal,2023,3,Recharge & bill payments,63055684,3.920985e+10
4132,west-bengal,2023,3,Financial Services,389751,4.249856e+08


In [11]:
mycursor.execute("""create table if not exists aggregated_transaction(State varchar(255),Year varchar(255),Quarter int,
                    Transaction_type varchar(255),Transaction_count bigint,Transaction_amount double)""")
for i,row in Agg_Trans.iterrows():
    sql = "insert into aggregated_transaction values(%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [12]:
path="pulse/data/aggregated/user/country/india/state/"
Agg_user_state_list=os.listdir(path)

clm={'State':[], 'Year':[],'Quater':[],'User_brand':[], 'User_count':[], 'User_percentage':[]}
for i in Agg_user_state_list:
    p_i=path+i+"/"
    Agg_yr=os.listdir(p_i)
    for j in Agg_yr:
        p_j=p_i+j+"/"
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+k
            Data=open(p_k,'r')
            E=json.load(Data)
            try:
                for z in E['data']['usersByDevice']:
                  brand = z['brand']
                  count = z['count']
                  percentage = (z['percentage']*100)
                  clm['User_brand'].append(brand)
                  clm['User_count'].append(count)
                  clm['User_percentage'].append(percentage)
                  clm['State'].append(i)
                  clm['Year'].append(j)
                  clm['Quater'].append(int(k.strip('.json')))
            except:
                  pass
Agg_user=pd.DataFrame(clm)

In [13]:
Agg_user

,State,Year,Quater,User_brand,User_count,User_percentage
0,andaman-&-nicobar-islands,2018,1,Xiaomi,1665,24.703264
1,andaman-&-nicobar-islands,2018,1,Samsung,1445,21.439169
2,andaman-&-nicobar-islands,2018,1,Vivo,982,14.569733
3,andaman-&-nicobar-islands,2018,1,Oppo,501,7.433234
4,andaman-&-nicobar-islands,2018,1,OnePlus,332,4.925816
...,...,...,...,...,...,...
6727,west-bengal,2022,1,Lenovo,330017,1.505567
6728,west-bengal,2022,1,Infinix,284678,1.298726
6729,west-bengal,2022,1,Asus,280347,1.278968
6730,west-bengal,2022,1,Apple,277752,1.267129


In [14]:
Agg_user.dtypes

State               object
Year                object
Quater               int64
User_brand          object
User_count           int64
User_percentage    float64
dtype: object

In [18]:
mycursor.execute("""create table if not exists aggregated_users(State varchar(255),Year varchar(255),Quarter int,
                    User_brand varchar(255),User_count bigint,User_percentage float) """)
for i,row in Agg_user.iterrows():
    sql = "insert into aggregated_users values(%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [15]:
path="pulse/data/map/transaction/hover/country/india/state/"
map_trans_state_list=os.listdir(path)

clm={'State':[], 'Year':[],'Quater':[],'District':[], 'Transacion_count':[], 'Transacion_amount':[]}
for i in map_trans_state_list:
    p_i=path+i+"/"
    Agg_yr=os.listdir(p_i)
    for j in Agg_yr:
        p_j=p_i+j+"/"
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+k
            Data=open(p_k,'r')
            D=json.load(Data)
            for z in D['data']['hoverDataList']:
              district = z['name'].split(' district')[0]
              count = z['metric'][0]['count']
              amount = z['metric'][0]['amount']
              clm['District'].append(district)
              clm['Transacion_count'].append(count)
              clm['Transacion_amount'].append(amount)
              clm['State'].append(i)
              clm['Year'].append(j)
              clm['Quater'].append(int(k.strip('.json')))
map_Trans=pd.DataFrame(clm)

In [16]:
map_Trans

,State,Year,Quater,District,Transacion_count,Transacion_amount
0,andaman-&-nicobar-islands,2018,1,north and middle andaman,442,9.316631e+05
1,andaman-&-nicobar-islands,2018,1,south andaman,5688,1.256025e+07
2,andaman-&-nicobar-islands,2018,1,nicobars,528,1.139849e+06
3,andaman-&-nicobar-islands,2018,2,north and middle andaman,825,1.317863e+06
4,andaman-&-nicobar-islands,2018,2,south andaman,9395,2.394824e+07
...,...,...,...,...,...,...
16827,west-bengal,2023,3,nadia,30146530,5.787869e+10
16828,west-bengal,2023,3,birbhum,17597047,3.090737e+10
16829,west-bengal,2023,3,purba medinipur,34244893,6.813718e+10
16830,west-bengal,2023,3,maldah,28539407,5.299766e+10


In [17]:
map_Trans.dtypes

State                 object
Year                  object
Quater                 int64
District              object
Transacion_count       int64
Transacion_amount    float64
dtype: object

In [19]:
mycursor.execute("""create table if not exists map_transaction(State varchar(255),Year varchar(255),Quarter int,
                 District varchar(255),Transaction_count bigint,Transaction_amount double)""")
for i,row in map_Trans.iterrows():
    sql = "insert into map_transaction values(%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [20]:
path="pulse/data/map/user/hover/country/india/state/"
map_user_state_list=os.listdir(path)

clm={'State':[], 'Year':[],'Quater':[],'District': [], 'Registered_user': [], 'App_opens': []}
for i in map_user_state_list:
    p_i=path+i+"/"
    Agg_yr=os.listdir(p_i)
    for j in Agg_yr:
        p_j=p_i+j+"/"
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+k
            Data=open(p_k,'r')
            E=json.load(Data)
            try:
                for z_key, z_value in E['data']['hoverData'].items():
                  district = z_key.split(' district')[0]
                  reg_user = z_value['registeredUsers']
                  app_opens = z_value['appOpens']
                  clm['District'].append(district)
                  clm['Registered_user'].append(reg_user)
                  clm['App_opens'].append(app_opens)
                  clm['State'].append(i)
                  clm['Year'].append(j)
                  clm['Quater'].append(int(k.strip('.json')))
            except:
                  pass
map_user=pd.DataFrame(clm)

In [21]:
map_user

,State,Year,Quater,District,Registered_user,App_opens
0,andaman-&-nicobar-islands,2018,1,north and middle andaman,632,0
1,andaman-&-nicobar-islands,2018,1,south andaman,5846,0
2,andaman-&-nicobar-islands,2018,1,nicobars,262,0
3,andaman-&-nicobar-islands,2018,2,north and middle andaman,911,0
4,andaman-&-nicobar-islands,2018,2,south andaman,8143,0
...,...,...,...,...,...,...
16831,west-bengal,2023,3,nadia,1544294,12674328
16832,west-bengal,2023,3,birbhum,974363,18961683
16833,west-bengal,2023,3,purba medinipur,1530289,18593769
16834,west-bengal,2023,3,maldah,1091634,28020285


In [22]:
map_user.dtypes

State              object
Year               object
Quater              int64
District           object
Registered_user     int64
App_opens           int64
dtype: object

In [23]:
mycursor.execute("""create table if not exists map_users(State varchar(255),Year varchar(255),Quarter int,
                    District varchar(255),Registered_user bigint,App_opens bigint) """)
for i,row in map_user.iterrows():
    sql = "insert into map_users values(%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [24]:
path="pulse/data/top/transaction/country/india/state/"
top_trans_state_list=os.listdir(path)

clm={'State':[], 'Year':[],'Quater':[],'District':[], 'Transacion_count':[], 'Transacion_amount':[]}
for i in top_trans_state_list:
    p_i=path+i+"/"
    Agg_yr=os.listdir(p_i)
    for j in Agg_yr:
        p_j=p_i+j+"/"
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+k
            Data=open(p_k,'r')
            D=json.load(Data)
            for z in D['data']['districts']:
              districts = z['entityName']
              count = z['metric']['count']
              amount = z['metric']['amount']
              clm['District'].append(districts)
              clm['Transacion_count'].append(count)
              clm['Transacion_amount'].append(amount)
              clm['State'].append(i)
              clm['Year'].append(j)
              clm['Quater'].append(int(k.strip('.json')))
top_Trans_districts=pd.DataFrame(clm)

In [25]:
top_Trans_districts

,State,Year,Quater,District,Transacion_count,Transacion_amount
0,andaman-&-nicobar-islands,2018,1,south andaman,5688,1.256025e+07
1,andaman-&-nicobar-islands,2018,1,nicobars,528,1.139849e+06
2,andaman-&-nicobar-islands,2018,1,north and middle andaman,442,9.316631e+05
3,andaman-&-nicobar-islands,2018,2,south andaman,9395,2.394824e+07
4,andaman-&-nicobar-islands,2018,2,nicobars,1120,3.072437e+06
...,...,...,...,...,...,...
6803,west-bengal,2023,3,hooghly,32021392,5.624223e+10
6804,west-bengal,2023,3,howrah,31734226,5.800237e+10
6805,west-bengal,2023,3,paschim bardhaman,30585672,4.340349e+10
6806,west-bengal,2023,3,paschim medinipur,30418455,5.043177e+10


In [26]:
top_Trans_districts.dtypes

State                 object
Year                  object
Quater                 int64
District              object
Transacion_count       int64
Transacion_amount    float64
dtype: object

In [27]:
mycursor.execute("""create table if not exists top_transaction_district(State varchar(255),Year varchar(255),Quarter int,
                    District varchar(255),Transaction_count bigint,Transaction_amount double)""")
for i,row in top_Trans_districts.iterrows():
    sql = "insert into top_transaction_district values(%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [29]:
path="pulse/data/top/transaction/country/india/state/"
top_pincode_trans_state_list=os.listdir(path)
top_pincode_trans_state_list

clm={'State':[], 'Year':[],'Quater':[],'Pincode':[], 'Transacion_count':[], 'Transacion_amount':[]}
for i in top_pincode_trans_state_list:
    p_i=path+i+"/"
    Agg_yr=os.listdir(p_i)
    for j in Agg_yr:
        p_j=p_i+j+"/"
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+k
            Data=open(p_k,'r')
            D=json.load(Data)
            for z in D['data']['pincodes']:
              pincodes = z['entityName']
              count = z['metric']['count']
              amount = z['metric']['amount']
              clm['Pincode'].append(pincodes)
              clm['Transacion_count'].append(count)
              clm['Transacion_amount'].append(amount)
              clm['State'].append(i)
              clm['Year'].append(j)
              clm['Quater'].append(int(k.strip('.json')))
top_Trans_pincode=pd.DataFrame(clm)

In [30]:
top_Trans_pincode

,State,Year,Quater,Pincode,Transacion_count,Transacion_amount
0,andaman-&-nicobar-islands,2018,1,744101,1622,2.769298e+06
1,andaman-&-nicobar-islands,2018,1,744103,1223,2.238042e+06
2,andaman-&-nicobar-islands,2018,1,744102,969,3.519060e+06
3,andaman-&-nicobar-islands,2018,1,744105,685,1.298561e+06
4,andaman-&-nicobar-islands,2018,1,744104,340,1.039715e+06
...,...,...,...,...,...,...
8205,west-bengal,2023,3,700001,3536928,7.193522e+09
8206,west-bengal,2023,3,721301,3464729,4.871566e+09
8207,west-bengal,2023,3,700039,3405551,5.095825e+09
8208,west-bengal,2023,3,732125,3347165,5.353831e+09


In [31]:
top_Trans_pincode.dtypes

State                 object
Year                  object
Quater                 int64
Pincode               object
Transacion_count       int64
Transacion_amount    float64
dtype: object

In [32]:
mycursor.execute("""create table if not exists top_transaction_pincode(State varchar(255),Year varchar(255),Quarter int,
                    Pincode varchar(255),Transaction_count bigint,Transaction_amount double) """)
for i,row in top_Trans_pincode.iterrows():
    sql = "insert into top_transaction_pincode values(%s,%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [37]:
path="pulse/data/top/user/country/india/state/"
top_user_state_list=os.listdir(path)

clm={'State':[], 'Year':[],'Quater':[],'District': [], 'Registered_user': []}
for i in top_user_state_list:
    p_i=path+i+"/"
    Agg_yr=os.listdir(p_i)
    for j in Agg_yr:
        p_j=p_i+j+"/"
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+k
            Data=open(p_k,'r')
            D=json.load(Data)
            for z in D['data']['districts']:
              districts = z['name']
              reg_user = z['registeredUsers']
              clm['District'].append(districts)
              clm['Registered_user'].append(reg_user)
              clm['State'].append(i)
              clm['Year'].append(j)
              clm['Quater'].append(int(k.strip('.json')))
top_user_districts=pd.DataFrame(clm)

In [38]:
top_user_districts

,State,Year,Quater,District,Registered_user
0,andaman-&-nicobar-islands,2018,1,south andaman,5846
1,andaman-&-nicobar-islands,2018,1,north and middle andaman,632
2,andaman-&-nicobar-islands,2018,1,nicobars,262
3,andaman-&-nicobar-islands,2018,2,south andaman,8143
4,andaman-&-nicobar-islands,2018,2,north and middle andaman,911
...,...,...,...,...,...
6803,west-bengal,2023,3,howrah,1608895
6804,west-bengal,2023,3,nadia,1544294
6805,west-bengal,2023,3,purba medinipur,1530289
6806,west-bengal,2023,3,paschim medinipur,1384076


In [39]:
top_user_districts.dtypes

State              object
Year               object
Quater              int64
District           object
Registered_user     int64
dtype: object

In [41]:
mycursor.execute("""create table if not exists top_users_district(State varchar(255),Year varchar(255),
                    Quarter int,District varchar(255),Registered_user bigint) """)
for i,row in top_user_districts.iterrows():
    sql = "insert into top_users_district values(%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()

In [42]:
path="pulse/data/top/user/country/india/state/"
top_pincode_user_state_list=os.listdir(path)

clm={'State':[], 'Year':[],'Quater':[],'Pincode': [], 'Registered_user': []}
for i in top_pincode_user_state_list:
    p_i=path+i+"/"
    Agg_yr=os.listdir(p_i)
    for j in Agg_yr:
        p_j=p_i+j+"/"
        Agg_yr_list=os.listdir(p_j)
        for k in Agg_yr_list:
            p_k=p_j+k
            Data=open(p_k,'r')
            D=json.load(Data)
            for z in D['data']['pincodes']:
              pincodes = z['name']
              reg_user = z['registeredUsers']
              clm['Pincode'].append(pincodes)
              clm['Registered_user'].append(reg_user)
              clm['State'].append(i)
              clm['Year'].append(j)
              clm['Quater'].append(int(k.strip('.json')))
top_user_pincode=pd.DataFrame(clm)

In [43]:
top_user_pincode

,State,Year,Quater,Pincode,Registered_user
0,andaman-&-nicobar-islands,2018,1,744103,1608
1,andaman-&-nicobar-islands,2018,1,744101,1108
2,andaman-&-nicobar-islands,2018,1,744105,1075
3,andaman-&-nicobar-islands,2018,1,744102,1006
4,andaman-&-nicobar-islands,2018,1,744104,272
...,...,...,...,...,...
8206,west-bengal,2023,3,700015,122316
8207,west-bengal,2023,3,742304,119245
8208,west-bengal,2023,3,721101,118798
8209,west-bengal,2023,3,700150,114987


In [44]:
top_user_pincode.dtypes

State              object
Year               object
Quater              int64
Pincode            object
Registered_user     int64
dtype: object

In [46]:
mycursor.execute("""create table if not exists top_users_pincode(State varchar(255),Year varchar(255),
                    Quarter int,Pincode varchar(255),Registered_user bigint) """)
for i,row in top_user_pincode.iterrows():
    sql = "insert into top_users_pincode values(%s,%s,%s,%s,%s)"
    mycursor.execute(sql, tuple(row))
    mydb.commit()
